In [1]:
import glob
import sqlite3

In [2]:
con = sqlite3.connect("/data/tables/processing.db")
cur = con.cursor()

In [3]:
cur.execute("""
SELECT product_id, last_processed FROM downloaded
WHERE last_processed IS NULL
""")

cur.fetchall()

[('98100297', None), ('98100103', None)]

In [4]:
cur.execute("""
SELECT count(*) FROM downloaded
""")
cur.fetchall()

[(7889,)]

In [5]:
cur.execute("""
SELECT DISTINCT product_id FROM downloaded
""")

product_ids_processed = [x[0] for x in cur.fetchall()]
print(len(product_ids_processed))

7889


# This is the entire productIds universe according to Statitics Canada

In [6]:
cur.execute("""
SELECT DISTINCT product_id FROM cubes
""")

print(len(cur.fetchall()))

7917


# The remaining productIds that I need to download

In [8]:
cur.execute("""
SELECT DISTINCT product_id FROM cubes
WHERE product_id NOT IN (SELECT product_id FROM downloaded)
""")

to_download = [x[0] for x in cur.fetchall()]

In [14]:
from datetime import datetime
import glob
from multiprocessing import Pool
import json
import os
import sqlite3
import zipfile
from zoneinfo import ZoneInfo

import pandas as pd
import requests
from tqdm import tqdm

data_folder = "/data/tables"
input_folder = f"{data_folder}/input"
scratch_folder = f"{data_folder}/scratch"
output_folder = f"{data_folder}/output"


def download_cube(product_id, language="en"):
    """
    Downloads the English CSV for a specific table
    """
    download_url = f"https://www150.statcan.gc.ca/t1/wds/rest/getFullTableDownloadCSV/{product_id}/en"
    response = requests.get(download_url).json()
    zip_url = response['object']
    zip_file_name = f"{input_folder}/{language}/june_25_2025/{product_id}.zip"
    print(f"Downloading {zip_url} to {zip_file_name}")
    response = requests.get(zip_url, stream=True, headers={"user-agent": None})
    progress_bar = tqdm(
        desc=zip_file_name,
        total=int(response.headers.get("content-length", 0)),
        unit="B",
        unit_scale=True
    )
    with open(zip_file_name, "wb") as handle:
        for chunk in response.iter_content(chunk_size=512):
            if chunk:  # filter out keep-alive new chunks
                handle.write(chunk)
                progress_bar.update(len(chunk))
        progress_bar.close()

In [15]:
for product_id in to_download:
    download_cube(product_id)

/data/tables/input/en/june_25_2025/12100147.zip: 100%|█████████████████| 312M/312M [07:33<00:00, 688kB/s]


/data/tables/input/en/june_25_2025/12100148.zip: 10


/data/tables/input/en/june_25_2025/12100149.zip: 100%|██████████████| 1.42G/1.42G [14:39<00:00, 1.62MB/s]


/data/tables/input/en/june_25_2025/12100150.zip: 100%|████████████████| 317M/317M [04:48<00:00, 1.10MB/s]


/data/tables/input/en/june_25_2025/12100151.zip: 100%|██████████████| 2.13G/2.13G [24:42<00:00, 1.43MB/s]


...

/data/tables/input/en/june_25_2025/38100182.zip: 100%|█████████████████| 403k/403k [00:00<00:00, 937kB/s]


/data/tables/input/en/june_25_2025/38100183.zip: 100%|████████████████| 105k/105k [00:00<00:00, 2.88MB/s]


/data/tables/input/en/june_25_2025/45100110.zip: 100%|███████████████| 16.2M/16.2M [00:19<00:00, 817kB/s]


/data/tables/input/en/june_25_2025/45100111.zip: 100%|███████████████| 5.20M/5.20M [00:11<00:00, 435kB/s]


/data/tables/input/en/june_25_2025/46100092.zip: 100%|██████████████| 73.9k/73.9k [00:00<00:00, 1.90MB/s]


# The remaining productIds that I need to process from input data directory

In [9]:
remaining_to_process = []
for file in glob.glob("/data/tables/input/en/*.zip"):
    product_id = file.split("/")[-1].split(".zip")[0]
    if product_id not in product_ids_processed:
        remaining_to_process.append(product_id)

In [10]:
print(len(remaining_to_process))

1


In [11]:
print(remaining_to_process)

['13100442']
